# Fix_corrupt_output
Fix a corrupt variable in a daily file by replacing it with the monthly mean times the number of days, minus the sum of the other dailies

In [1]:
import xarray as xr
from glob import glob
from shutil import copyfile
from datetime import timedelta

In [2]:
badvar = 'frazil'  # name of corrupt var
monthsuffix = '_m'  # suffix for monthly variables (eg for CICE data)

mfile = '/scratch/x77/aek156/access-om2/archive/01deg_jra55v140_iaf_cycle3/output518/ice/OUTPUT/iceh.1965-08.nc'  # good monthly mean file

fpath = '/scratch/v45/aek156/access-om2/archive/01deg_jra55v140_iaf_cycle3/output518/ice/OUTPUT/'  # path to daily files
prevfile = 'iceh.1965-08-16.nc'  # file before badfile
badfile = 'iceh.1965-08-17.nc'  # file with corrupt variable "badvar" -- now renamed to iceh.1965-08-17.nc-CORRUPT-frazil
fixedfn = badfile+'-fixed-'+badvar # to be created by this script

In [3]:
goodfiles = [x for x in glob(fpath+'iceh.1965-08-??.nc') if badfile not in x]
goodfiles.sort()

In [4]:
# calculate replacement value of badvar in badfile required to make the monthly mean match that in mfile
prevvar = xr.open_dataset(fpath+prevfile)[badvar]
fixedvar = prevvar  # start with previous day's dataarray

fixedvar.data = (len(goodfiles) + 1)*xr.open_dataset(mfile)[badvar+monthsuffix].data  #rename(badvar)

for f in goodfiles:
    fixedvar.data -= xr.open_dataset(f)[badvar].data

fixedvar = fixedvar.clip(min=0.0)  # ensure values are non-negative (which can happen due to roundoff)
fixedvar = fixedvar.where(prevvar.notnull(), prevvar.encoding['_FillValue'])  # reinstate mask from prevvar
fixedvar.attrs = prevvar.attrs
fixedvar.encoding = prevvar.encoding
fixedvar.encoding['source'] = fpath+badfile

# now put fixedvar into .nc file
fixedds = xr.open_dataset(fpath+badfile, drop_variables=badvar)
fixedvar['time'] = fixedds['time']
fixedds = eval('fixedds.assign('+badvar+'=fixedvar)')

# there are some attribute differences in time and time_bounds but hopefully they don't matter too much

fixedds.to_netcdf(fpath+fixedfn)